# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
file="../output_data/cities(student).csv"
weather_df=pd.DataFrame(pd.read_csv(file))
weather_df

,Unnamed: 0,City,Cloudiness,Country,Latitude,Longitude,Humidity,Max Temp,Wind Speed
0,0,majene,100.0,ID,-3.5403,118.9707,91.0,24.59,3.39
1,1,brae,100.0,GB,60.3964,-1.3530,85.0,11.95,5.66
2,2,esperance,100.0,AU,-33.8667,121.9000,68.0,13.42,4.93
3,3,yellowknife,60.0,CA,62.4560,-114.3525,93.0,16.85,3.58
4,4,thompson,18.0,CA,55.7435,-97.8558,97.0,15.94,2.15
...,...,...,...,...,...,...,...,...,...
553,603,gallup,100.0,US,35.5281,-108.7426,73.0,21.42,0.45
554,604,egvekinot,100.0,RU,66.3167,-179.1667,86.0,5.06,4.34
555,605,halifax,100.0,CA,44.6453,-63.5724,94.0,19.67,0.45
556,606,miraflores,99.0,CO,19.2167,-98.8083,92.0,16.13,0.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
weather_location=weather_df[["Latitude","Longitude"]]
humidity=weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(layout={'width':'900px','height':'750px'})

# Create heat layer
heat_layer = gmaps.heatmap_layer(weather_location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='750px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 25 degrees (C) but higher than 10 degrees (C).
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

vacation_df=weather_df.loc[(weather_df["Max Temp"]<25)&(weather_df["Max Temp"]>10)&(weather_df["Wind Speed"]<10)&(weather_df["Cloudiness"]==0)]

vacation_df=vacation_df.reset_index()

#check that the number of samples are equal to 46
print(len(vacation_df))

vacation_df.head()

46


,index,Unnamed: 0,City,Cloudiness,Country,Latitude,Longitude,Humidity,Max Temp,Wind Speed
0,19,21,rikitea,0.0,PF,-23.1203,-134.9692,71.0,22.30,6.92
1,23,25,emerald,0.0,AU,-23.5333,148.1667,27.0,16.86,4.18
2,46,49,yafran,0.0,LY,32.0633,12.5286,43.0,24.72,2.02
3,48,51,halle,0.0,DE,51.5000,12.0000,66.0,21.76,1.54
4,65,70,tadine,0.0,NC,-21.5500,167.8833,75.0,21.18,2.38


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
hotel_df=pd.DataFrame(vacation_df[["City","Country", "Latitude", "Longitude"]])

hotel_df["Hotel Name"]=""
hotel_df["Hotel Lat"]=""
hotel_df["Hotel Lng"]=""
hotel_df["Hotel Address"]=""

hotel_df=hotel_df.rename(columns={"Latitude": "City Latitude", "Longitude": "City Longitude"})

hotel_df.head()

,City,Country,City Latitude,City Longitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address
0,rikitea,PF,-23.1203,-134.9692,,,,
1,emerald,AU,-23.5333,148.1667,,,,
2,yafran,LY,32.0633,12.5286,,,,
3,halle,DE,51.5000,12.0000,,,,
4,tadine,NC,-21.5500,167.8833,,,,


In [129]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    # "location":"-31.993362597635887, 115.86432959807914",
    "radius":"5000",
    "keyword": "hotel",
    "type":"lodging",
    "key": g_key}

# response=requests.get(base_url, params=params).json()

# print(json.dumps(response["results"][0]["vicinity"],indent=4, sort_keys=True))

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    #define city lat & lng
    city_lat=row["City Latitude"]
    city_lng=row["City Longitude"]

    # add keyword to params dict
    params['location'] = f"{city_lat},{city_lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index} for City {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # # extract results
    # results = response['results']
    try:
        print(f"Closest hotel in the vicinity is {response['results'][0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
        hotel_df.loc[index, 'Hotel Address'] = response["results"][0]["vicinity"]
        hotel_df.loc[index, 'Hotel Lat'] = response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Hotel Lng'] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0 for City rikitea.
Closest hotel in the vicinity is People ThankYou.
------------
Retrieving Results for Index 1 for City emerald.
Closest hotel in the vicinity is Emerald Maraboon Motor Inn.
------------
Retrieving Results for Index 2 for City yafran.
Closest hotel in the vicinity is فندق يفرن السياحي.
------------
Retrieving Results for Index 3 for City halle.
Closest hotel in the vicinity is the niu Ridge.
------------
Retrieving Results for Index 4 for City tadine.
Missing field/result... skipping.
------------
Retrieving Results for Index 5 for City ponta do sol.
Closest hotel in the vicinity is Palace of Zinos.
------------
Retrieving Results for Index 6 for City sao borja.
Closest hotel in the vicinity is Al-Manara Hotel.
------------
Retrieving Results for Index 7 for City walvis bay.
Closest hotel in the vicinity is Protea Hotel by Marriott Walvis Bay Pelican Bay.
------------
Retrieving Results for Index 8 for City caconda.
Missing field/result..

In [132]:
#note that Hotel Lat & Hotel Lng Column is currently treated as an object as there are some samples that came back with no results. remove these samples and it will fix the issue
hotel_df=hotel_df.loc[hotel_df["Hotel Name"]!=""]

# #alternatively, convert hotel Lat & Lng into float. code is sourced from https://stackoverflow.com/questions/8420143/valueerror-could-not-convert-string-to-float-id 
# # hotel_df["Hotel Lat"]=hotel_df["Hotel Lat"].replace('','').astype(float)
# # hotel_df["Hotel Lng"]=hotel_df["Hotel Lng"].replace('','').astype(float)


hotel_df.dtypes

City               object
Country            object
City Latitude     float64
City Longitude    float64
Hotel Name         object
Hotel Lat         float64
Hotel Lng         float64
Hotel Address      object
dtype: object

In [133]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [136]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='750px', width='900px'))